In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim

In [3]:
#Load Data and Perform some feature transformations
df=pd.read_csv("weatherdata.csv")
df.head()

,time,tavg,tmin,tmax,prcp,wspd,pres
0,2014-01-01,13.1,8.3,20.0,0.0,5.4,1019.1
1,2014-01-02,14.4,8.3,21.7,0.0,4.3,1017.9
2,2014-01-03,14.2,7.8,18.9,0.0,3.2,1015.7
3,2014-01-04,15.4,13.3,18.9,0.0,2.9,1015.6
4,2014-01-05,15.4,10.0,23.3,0.0,5.0,1018.8


In [4]:
df=df.drop(['tmin','tmax'],axis=1)
df.head()

,time,tavg,prcp,wspd,pres
0,2014-01-01,13.1,0.0,5.4,1019.1
1,2014-01-02,14.4,0.0,4.3,1017.9
2,2014-01-03,14.2,0.0,3.2,1015.7
3,2014-01-04,15.4,0.0,2.9,1015.6
4,2014-01-05,15.4,0.0,5.0,1018.8


In [6]:
df['time']=pd.to_datetime(df['time'])
df['month']=df['time'].dt.month
df.head()

,time,tavg,prcp,wspd,pres,month
0,2014-01-01,13.1,0.0,5.4,1019.1,1
1,2014-01-02,14.4,0.0,4.3,1017.9,1
2,2014-01-03,14.2,0.0,3.2,1015.7,1
3,2014-01-04,15.4,0.0,2.9,1015.6,1
4,2014-01-05,15.4,0.0,5.0,1018.8,1


In [9]:
#prepare data for the model
X=df[['month','prcp','wspd','pres']].values
y=df['tavg'].values

Xtrain,Xtest,ytrain,ytest=train_test_split(X,y,test_size=0.2,random_state=20)

In [11]:
scaler=StandardScaler()
Xtrain=scaler.fit_transform(Xtrain)
Xtest=scaler.fit_transform(Xtest)

In [12]:
Xtrain=torch.tensor(Xtrain)
Xtest=torch.tensor(Xtest)
ytrain=torch.tensor(ytrain).view(-1,1)
ytest=torch.tensor(ytest).view(-1,1)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "C:\Users\ayman\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\ayman\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "d:\DeepLearningProject\.venv\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "d:\DeepLearningProject\.venv\lib\site-packages\traitlets\config\application.py", line 1075, in launch_in

In [13]:
Xtrain=Xtrain.view(Xtrain.size(0),1,Xtrain.size(1))
Xtest=Xtest.view(Xtest.size(0),1,Xtest.size(1))


In [14]:
#Model Definition

class RNN(nn.Module):
    def __init__(self,inputsize,hiddensize,numlayers,outputsize):
        super(RNN,self).__init__()
        self.rnn=nn.RNN(inputsize,hiddensize,numlayers,batch_first=True)
        self.fc=nn.Linear(hiddensize,outputsize)

    def forward(self,x):
        h0=torch.zeros(numlayers,x.size(0),hiddensize).to(x.device)

        out,_=self.rnn(x,h0)
        out=self.fc(out[:,-1,:])

        return out
    
inputsize=4
hiddensize=64
numlayers=1
outputsize=1

model=RNN(inputsize,hiddensize,numlayers,outputsize)

criterion=nn.MSELoss()
optimizer=optim.Adam(model.parameters(),lr=0.001)

In [16]:
#training loop
numepochs=1000
for epoch in range(numepochs):
    model.train()
    optimizer.zero_grad()
    outputs=model(Xtrain)
    loss=criterion(outputs,ytrain)
    loss.backward()
    optimizer.step()
    print("epoch :",epoch,"loss :",loss.item())
    

epoch : 0 loss : 349.1927490234375
epoch : 1 loss : 348.65728759765625
epoch : 2 loss : 348.122802734375
epoch : 3 loss : 347.5892028808594
epoch : 4 loss : 347.05645751953125
epoch : 5 loss : 346.5244445800781
epoch : 6 loss : 345.99310302734375
epoch : 7 loss : 345.4621276855469
epoch : 8 loss : 344.9311828613281
epoch : 9 loss : 344.40008544921875
epoch : 10 loss : 343.86846923828125
epoch : 11 loss : 343.33599853515625
epoch : 12 loss : 342.8024597167969
epoch : 13 loss : 342.2675476074219
epoch : 14 loss : 341.73095703125
epoch : 15 loss : 341.1923522949219
epoch : 16 loss : 340.6513977050781
epoch : 17 loss : 340.1077575683594
epoch : 18 loss : 339.56121826171875
epoch : 19 loss : 339.0113220214844
epoch : 20 loss : 338.457763671875
epoch : 21 loss : 337.9002990722656
epoch : 22 loss : 337.33856201171875
epoch : 23 loss : 336.77227783203125
epoch : 24 loss : 336.20111083984375
epoch : 25 loss : 335.6247863769531
epoch : 26 loss : 335.0429992675781
epoch : 27 loss : 334.4554138183

In [17]:
#test loop
model.eval()
with torch.no_grad():
    predictions=model(Xtest)
    testloss=criterion(predictions,ytest)
    print("loss :",testloss.item())

loss : 4.526377201080322
